In [1]:
import cPickle as pickle
import matplotlib.pylab as pl
%matplotlib inline
import numpy as np
import mxnet as mx
import mx_layers as layers

In [2]:
n_residual_layers = 4

In [3]:
accuracy, progress = pickle.load(open('info/residual-network-on-shrinked-mnist-4', 'rb'))
accuracy

IOError: [Errno 2] No such file or directory: 'info/residual-network-on-shrinked-mnist-4'

In [ ]:
parameters, states = pickle.load(open('parameters/residual-network-on-shrinked-mnist-4', 'rb'))

In [ ]:
def _normalized_convolution(index, **kwargs):
  network = layers.convolution(no_bias=True, name='convolution%d' % index, **kwargs)
  network = layers.batch_normalization(network, fix_gamma=False, name='batchnorm%d' % index)
  network = layers.ReLU(network)
  return network

In [ ]:
kwargs = {'n_filters' : 16, 'kernel_shape' : (5, 5), 'stride' : (1, 1), 'pad' : (2, 2)}
network = layers.variable('data')
for index in range(3):
  network = _normalized_convolution(index, X=network, **kwargs)
  network = layers.pooling(X=network, mode='maximum', kernel_shape=(2, 2), stride=(2, 2), pad=(0, 0))

In [ ]:
def feature_map_variance(feature_maps):
    N, C, W, H = feature_maps.shape
    feature_maps = feature_maps.reshape((N, C, W * H))
    mean = mx.nd.mean(feature_maps, axis=2, keepdims=True)
    variance = mx.nd.mean((feature_maps - mean) ** 2, axis=1, keepdims=True)
    return mx.nd.mean(variance)

In [ ]:
from data_utilities import load_mnist
stretched_mnist = load_mnist(path='stretched_mnist', scale=1, shape=(1, 56, 56))
stretched_canvas_mnist = load_mnist(path='stretched_canvas_mnist', scale=1, shape=(1, 56, 56))

In [ ]:
context = mx.cpu()
args = {key : mx.nd.array(value, context) for key, value in parameters.items()}
aux_states = {key : mx.nd.array(value, context) for key, value in states.items()}

In [ ]:
N = 1000
args['data'] = mx.nd.array(stretched_mnist[4], context)[:N]
executor = network.bind(context, args, aux_states=aux_states)
scores = executor.forward()[0]
score_variance(scores).asscalar()

In [ ]:
args['data'][:] = mx.nd.array(stretched_canvas_mnist[4], context)[:N]
scores = executor.forward()[0]
score_variance(scores).asscalar()